In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils


ROOT_DIR = "C:\\Users\\NJ\\MaskRCNN-Folder" # change path and insert your downloaded files folder path

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")


WEIGHTS_PATH = "C:/Users/NJ/MASKRCNN-Folder/logs/object20211105T2126/mask_rcnn_object_0020.h5"   # change address path and insert your weights file path


In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    NAME = "object"

    IMAGES_PER_GPU = 1

    NUM_CLASSES = 1 + 1  # Background + mosquito

    STEPS_PER_EPOCH = 10

    # Skip detections with < 70% confidence
    DETECTION_MIN_CONFIDENCE = 0.7

In [ ]:
config = CustomConfig()
CUSTOM_DIR = os.path.join(ROOT_DIR, "/dataset/")
class InferenceConfig(config.__class__):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.7

config = InferenceConfig()
config.display()

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "mosquito")
      
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        annotations1 = json.load(open(os.path.join(dataset_dir, "via_project.json")))

        annotations = list(annotations1.values())  

        annotations = [a for a in annotations if a['regions']]
        
        for a in annotations:
           
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"mosquito": 1}
            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object", 
                image_id=a['filename'],  
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
       
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):

        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids 

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [ ]:
TEST_MODE = "inference"
ROOT_DIR = "C:\\Users\\NJ\\MASKRCNN-Folder\\dataset" # change it and provide path to to your data set folder

def get_ax(rows=1, cols=1, size=16):
  """Return a Matplotlib Axes array to be used in all visualizations in the notebook.  Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax


CUSTOM_DIR = "C:/Users/NJ/MASKRCNN-Folder/dataset"  # change it and provide path to to your data set folder
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

In [ ]:
weights_path = WEIGHTS_PATH

print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
image_id = random.choice(dataset.image_ids)
print("image id is :",image_id)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,dataset.image_reference(image_id)))

results = model.detect([image], verbose=1)

In [ ]:
class_names = [
    'BG', 'mosquito', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
    'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
    'teddy bear', 'hair drier', 'toothbrush', 'person'
]

In [ ]:
def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}

In [ ]:
def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

In [ ]:
def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)

        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

    return image

In [ ]:
if __name__ == '__main__':
    """
        test everything
    """

    capture = cv2.VideoCapture('Video.mp4')

    frame_width = int(capture.get(3))
    frame_height = int(capture.get(4))
    out = cv2.VideoWriter('mask_Video.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 60, (frame_width,frame_height))

    count = 1
    aa = 0
    ab = 1

    while True:
        ret, frame = capture.read()
        results = model.detect([frame], verbose=0)
        r = results[0]
        frame = display_instances(
            frame, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores']
        )
        frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
        out.write(frame)
        A = (r['rois'])
        f = open(f"x1{count}.txt", 'a')
        k = open(f"y1{count}.txt", 'a')
        fcount = open(f"x1count{count}.txt", 'a')
        kcount = open(f"y1count{count}.txt", 'a')
        f.truncate(0)
        k.truncate(0)
        for i in A:
            g='g'
            h='h'
            h+=str(i)
            g+=str(i)
            g = (r['rois'][aa][0]+r['rois'][aa][2])/2
            h = (r['rois'][aa][1]+r['rois'][aa][3])/2
            countnumber=count
            f.write(f"{h}\n")
            k.write(f"{g}\n")
            fcount.write(f"{h} {countnumber}\n")
            kcount.write(f"{g} {countnumber}\n")
            aa += 1 
        aa = 0
        ab = 1
        f.close()
        k.close()
        fcount.close()
        kcount.close()
        fpx = open(f"x1{count}.txt", 'r')
        fpxstring = fpx.read()
        fpxstring = fpxstring[:-1]
        fpxlist = fpxstring.split("\n")
        for i in range(0, len(fpxlist), 1):
            fpxlist[i] = fpxlist[i].replace(",","")
            fpxlist[i] = float(fpxlist[i])
        
        fpy = open(f"y1{count}.txt", 'r')
        fpystring = fpy.read()
        fpystring = fpystring[:-1]
        fpylist = fpystring.split("\n")
        for i in range(0, len(fpylist), 1):
            fpylist[i] = fpylist[i].replace(",","")
            fpylist[i] = float(fpylist[i])
        fpx.close()
        fpy.close()
        from math import sqrt

        if (count==1):
            ci = 1
        else:
            pass
        c = 0.05

        l = []

        ox = []
        yx = []
        for d in zip(fpxlist,fpylist):
            ss = 1
            c_rms = sqrt(d[0]**2+d[1]**2)
            for rms in l:
                if abs(c_rms/rms - 1) < c:
                    break
            else:
                l.append(c_rms)
                ox.append(d[0])
                yx.append(d[1])
                if (count==1):
                    f1 = open(f"x2{ci}.txt", 'a')
                    k1 = open(f"y2{ci}.txt", 'a')
                    f1count = open(f"x2count{ci}.txt", 'a')
                    k1count = open(f"y2count{ci}.txt", 'a')
                    f1.write(f"{d[0]}\n")
                    k1.write(f"{d[1]}\n")
                    f1count.write(f"{d[0]} {countnumber}\n")
                    k1count.write(f"{d[1]} {countnumber}\n")
                    f1.close()
                    k1.close()
                    f1count.close()
                    k1count.close()
                    ci +=1
                    ss +=1
                else:
                    continue
        if (count==1):
            ci -=1
        else: 
            pass
        cd = ci
        ce = ci
        ch = ci+1
        cm = ci
        cz = 0
        ciminus1 = ci-1
        if (count>=2):
            os.remove (f"x1{(count-1)}.txt")
            os.remove (f"y1{(count-1)}.txt")
            for s in zip(ox,yx):
                cc =15
                cd=ci+cz
                cm = ci+cz
                for i in range(0,cd,1):
                    f3 = open(f"x2{cd}.txt", 'r')
                    f3string = f3.read()
                    f3string = f3string[:-1]
                    f3list = f3string.split("\n")
                    for i in range(0, len(f3list), 1):
                        f3list[i] = f3list[i].replace(",","")
                        f3list[i] = float(f3list[i])
                    f3value = float(f3list[-1])
                    k3 = open(f"y2{cd}.txt", 'r')
                    k3string = k3.read()
                    k3string = k3string[:-1]
                    k3list = k3string.split("\n")
                    for j in range(0, len(k3list), 1):
                        k3list[j] = k3list[j].replace(",","")
                        k3list[j] = float(k3list[j])
                    k3value = float(k3list[-1])
                    drms = sqrt((k3value**2)+(f3value**2))
                    s0f3 = abs(s[0]-f3value)
                    s1k3 = abs(s[1]-k3value)
                    f3.close()
                    k3.close()
                    if ((s0f3<16) and (s1k3<16)):
                        cc -=1
                        f3 = open(f"x2{cd}.txt", 'a')
                        k3 = open(f"y2{cd}.txt", 'a')
                        f3count = open(f"x2count{cd}.txt", 'a')
                        k3count = open(f"y2count{cd}.txt", 'a')
                        f3.write(f"{s[0]}\n")
                        k3.write(f"{s[1]}\n")
                        f3count.write(f"{s[0]} frame number: {countnumber}\n")
                        k3count.write(f"{s[1]} frame number: {countnumber}\n")
                        f3.close()
                        k3.close()
                        f3count.close()
                        k3count.close()
                        cd -=1
                    else:
                        cd -=1
                if (cc==15):
                    cz += 1
                    f2 = open(f"x2{ch}.txt", 'a')
                    k2 = open(f"y2{ch}.txt", 'a')
                    f2count = open(f"x2count{ch}.txt", 'a')
                    k2count = open(f"y2count{ch}.txt", 'a')
                    f2.write(f"{s[0]}\n")
                    k2.write(f"{s[1]}\n")
                    f2count.write(f"{s[0]} {countnumber}\n")
                    k2count.write(f"{s[1]} {countnumber}\n")
                    f2.close()
                    k2.close()
                    f2count.close()
                    k2count.close()
                    ch +=1

        ch -=1
        print (f"Processed Frame Number: {count}")
        if (cm>ch):
            ci=cm
        elif (ch>cm):
            ci=ch
        else:
            ci=cm
        count += 1
        os.remove (f"x1count{(count-1)}.txt")
        os.remove (f"y1count{(count-1)}.txt")
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()